In [ ]:
%matplotlib inline

import numpy as np
import tensorflow as tf
import gym
import matplotlib.pyplot as plt
from IPython.display import clear_output

import gym_utils
from gen_raw_data import generate_action

import data_utils
from exp_parameters import ExpParam
from train_vision import create_or_load_vae

In [ ]:
################## SETTINGS #####################
# raw_dim = (210, 160, 3)
# net_dim = (32*4, 32*3, 3)

raw_dim = (84, 84, 4)
net_dim = (84, 84, 4)


x = 0
std = 0

exp_param = ExpParam(
    lat_type="discrete",
    dataset='breakout',
    name_prefix='_BLM'+str(x)+'_STD'+str(std),
    latent=[[32*128, 2]],
    raw_type=tf.uint8,
    raw_dim=raw_dim,
    net_dim=net_dim,  # very close to org aspect ration
    g_std=std,
    learning_rate=0.001,
    rec_loss_multiplier=x,
    max_example=5e6,
    batch_size=256,  # for testing
)

data_set = exp_param.dataset
batch_size = exp_param.batch_size

### DATA
train_iter, test_iter = data_utils.load_data(batch_size, data_set)
ball_col = data_utils.ball_col
ball_loss_multiplier = exp_param.rec_loss_multiplier


In [ ]:
# ################## SETTINGS #####################
# raw_dim = (210, 160, 3)
# net_dim = (32*4, 32*3, 3)

# x = 0
# std = 0.375

# exp_param = ExpParam(
#     lat_type="discrete",
#     dataset='breakout',
#     name_prefix='_BLM'+str(x)+'_STD'+str(std),
#     latent=[[32*128, 2]],
#     raw_type=tf.uint8,
#     raw_dim=raw_dim,
#     net_dim=net_dim,  # very close to org aspect ration
#     g_std=std,
#     learning_rate=0.001,
#     rec_loss_multiplier=x,
#     max_example=5e6,
#     batch_size=256,  # for testing
# )

# data_set = exp_param.dataset
# batch_size = exp_param.batch_size

# print(exp_param.g_std)
# print(exp_param.g_size)



In [ ]:
### GENERAL SETUP
try:
    sess.close()
except:
    pass

model_name = 'breakout_discrete_BLM64_STD0_LAT4096(2)_decent_perf'
model_path = './saves/'
model_path += model_name

model_name ='breakout_discrete_BLM1_STD0_LAT4096(2)_MADE1544445274'
model_path = './saved_model/'
model_path += model_name

model_name ='breakout_discrete_BLM64_STD0_lr0.0001_LAT4096(2)_MADE1543847099'
model_path = 'C:\\Users\\Toke\\Dropbox\\MAI\\'
model_path += model_name

print(model_path)

### NETWORK
sess, network, saver = create_or_load_vae(model_path, exp_param=exp_param)
# tf.global_variables_initializer().run()


In [ ]:
_, _, data = next(test_iter)
images = data[0]
# masks = data[1] * exp_param.rec_loss_multiplier
masks = data[1]  # TODO: Test that this works! (currently 'exp_param.rec_loss_multiplier' is multiplied twice'

# TODO: Test should use hard sample
[img, rec, sum] = sess.run([network.image, network.reconstructions, network.sum_img], feed_dict={
    network.raw_input: images,
    network.mask_in: masks,
    network.is_training: False
})


In [ ]:
img.shape

In [ ]:
from IPython.display import clear_output

plt.figure(figsize=(12,12))
for i in range(images.shape[0]):
# for i in range(5):
    file_name = 'breakout_render_{:05d}.png'.format(i)
    plt.suptitle(file_name)
    
    plt.subplot(241)
    plt.axis('off')
    plt.imshow(img[i,:,:,0])
    
    plt.subplot(242)
    plt.axis('off')
    plt.imshow(img[i,:,:,1])

    plt.subplot(245)
    plt.axis('off')
    plt.imshow(img[i,:,:,2])
    
    plt.subplot(246)
    plt.axis('off')
    plt.imshow(img[i,:,:,3])


    plt.subplot(243)
    plt.axis('off')
    plt.imshow(rec[i,:,:,0])
    
    plt.subplot(244)
    plt.axis('off')
    plt.imshow(rec[i,:,:,1])

    plt.subplot(247)
    plt.axis('off')
    plt.imshow(rec[i,:,:,2])
    
    plt.subplot(248)
    plt.axis('off')
    plt.imshow(rec[i,:,:,3])

    plt.axis('off')
#     plt.savefig('./media/images' + file_name)
    clear_output(wait=True)
    plt.show()
    

In [ ]:
## https://stackoverflow.com/questions/41228209/making-gif-from-images-using-imageio-in-python/45258744
import os
import imageio

png_dir = './media/images'
images = []
for file_name in os.listdir(png_dir):
    if file_name.endswith('.png'):
        file_path = os.path.join(png_dir, file_name)
        images.append(imageio.imread(file_path))
        
imageio.mimsave('./media/gif/movie.gif', images)
